* introduction

i used this dataset that collects information from 100k medical appointments in Brazil 

* Question

What factors are important for us to know in order to predict if a patient will show up for their scheduled appointment?

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

*   Data Wrangling

we will load the data and check for cleanliness.


In [2]:

#we will load the data and check for cleanliness
raw_df = pd.read_csv(r'../input/medical-appointment-no-shows/KaggleV2-May-2016.csv')
raw_df.head()

In [3]:
df = raw_df.copy()
df.shape

In [4]:
#No of missing value
df.info()

Fortunately, there is no missing values in dataset.



In [5]:
df.duplicated().sum()

In [6]:
df['PatientId'].nunique()

In [7]:

df['PatientId'].duplicated().sum()


There is 48228 duplicated PatientId 

In [8]:
df.describe()




* Data Cleaning


There is -1 years old in  an age column, which is impossible,so we will dorp it.

No-show column is confusing. Inorder to make it more clear, we can change the column name to 'Show', and change the object in this column. Some mistake in label name such as'Handcap' and 'Hipertension' should also be corrected.

If two rows has absolutely same information regardless of AppointmentID, we can treat them as duplicated information.
The data types of scheduled day and appointment day are str, which need to be transfered to datatime, in order to be analyzed easily.

In order to make in-depth analysis of relationship between appointment time and presence, some new columns show be added.


In [9]:
df.query('Age==-1')

In [10]:
df.drop(index=99832,inplace=True)
df.describe()


In [11]:
df.rename(columns={'Hipertension': 'Hypertension','Handcap': 'Handicap','No-show' : 'show'},inplace=	True)

df.head()

In [12]:
df['show']=df['show'].replace({'No':1,'Yes':0 })
df['show']=df['show'].astype('int')
df.head()


In [13]:
#check number of PatientId and show duplicated 
df.duplicated(['PatientId','show']).sum()

In [14]:
df.drop_duplicates(['PatientId','show'],inplace=True)
df.shape

In [15]:
#remove un importatnt data 
df.drop(['PatientId','AppointmentID','ScheduledDay','AppointmentDay'],axis= 1,inplace=True )
df.head()

* Exploratory Data Analysis

Now we ready to move on exploration 


In [16]:
df.hist(figsize=(12,10));

In [17]:
show_data = df.show == 1
noshow_data = df.show == 0
df[show_data].count(), df[noshow_data].count()

Number of showed patient(54153) is 3 times more than Non showed (17663)

In [18]:
df[show_data].mean(),df[noshow_data].mean()

showed patient recieved sms less than unshowed ones that means we have to check sms campaign 

In [19]:
# does Age affect the scheduled appointment?
def  appointment(df,col_name,attendance,absence):
    plt.figure(figsize=[12,6])
    df[col_name][show_data].hist(color= 'blue',label='show')
    df[col_name][noshow_data].hist(color='orange',label='noshow')
    plt.legend();
    plt.title('The relation between Age & scheduled Appointments');
    plt.xlabel('Age');
    plt.ylabel('No of Appointments');
appointment(df,'Age',show_data,noshow_data)

Ages from 0 ~ 9 are the most attending that means parents take care of their kids then  Ages from 45 ~ 55 and people > 65 years old are the least attending.

In [20]:
# does the Age and chronic diseases affect the scheduled appointment?

plt.figure(figsize=[12,6])
df[show_data].groupby(['Diabetes','Hypertension']).mean()['Age'].plot(kind= 'bar',color= 'blue', label= 'show')
df[noshow_data].groupby(['Diabetes','Hypertension']).mean()['Age'].plot(kind= 'bar',color= 'orange', label= 'noshow')
plt.title('The relation between chronic diseases & scheduled Appointments')
plt.legend();
plt.xlabel('chronic diseases')
plt.ylabel('average age');


In [21]:
df[show_data].groupby(['Diabetes','Hypertension']).mean()['Age'],df[noshow_data].groupby(['Diabetes','Hypertension']).mean()['Age']

There is no correlation between mean Age of chronic diseases & scheduled Appointments

In [22]:
# Percentage of people who show-up ?

def  appointment(df,col_name,attendance,absence):
    plt.figure(figsize=[12,6])
    df[col_name][show_data].value_counts(normalize=True).plot(kind='pie',label='show')
    plt.legend();
    plt.title('The relation between Gender & No of patients');
    plt.xlabel('Gender');
    plt.ylabel('No of patients');
appointment(df,'Gender',show_data,noshow_data)

In [23]:
# Percentage of people who show-up ?

def  appointment(df,col_name,attendance,absence):
    plt.figure(figsize=[12,6])
    df[col_name][noshow_data].value_counts(normalize=True).plot(kind='pie',label='noshow')
    plt.legend();
    plt.title('The relation between Gender & No of patients');
    plt.xlabel('Gender');
    plt.ylabel('No of patients');
appointment(df,'Gender',show_data,noshow_data)

there is no correlation between the Gender & scheduled Appointments

In [24]:
df[show_data].groupby(['Gender']).mean()['Age'],df[noshow_data].groupby(['Gender']).mean()['Age']


 there is no correlation between the average Age of Gender & scheduled Appointments

In [25]:
# does the Neighbourhood affects the Neighbourhood
plt.figure(figsize=[12,6])
df[show_data]['Neighbourhood'].value_counts().plot(kind= 'bar',color= 'blue', label= 'show')
df[noshow_data]['Neighbourhood'].value_counts().plot(kind= 'bar',color= 'orange', label= 'noshow')
plt.title('The relation between Neighbourhood & scheduled Appointments')
plt.legend();
plt.xlabel('Neighbourhood')
plt.ylabel('no of patients');

there is a correlation between the Neighbourhood & scheduled Appointments

* Conclusions

After making some questions on the dataset , we figured out some information about the patients and their behavior through questions as in the last question we found out that JARDIM CAMBURI seems to has most attending patients , in first  question we found out Ages from 0 ~ 9 are the most attending that means parents take care of their kids then  Ages from 45 ~ 55 and people > 65 years old are the least attending.
for a nother question There is no correlation between mean Age of chronic diseases & scheduled Appointments.



* Limitations

There is no a clear correlation between Age & Choronic diseases and showing appointment
